#### This code was used to integrate the drivers in the lead-up to our MHW events. It is VERY computationally expensive (I did loops, it was not fully vectorised). Make sure that you have over 600 GB of RAM handy!

In [44]:
#Load required packages
%matplotlib inline
import matplotlib.pyplot as plt
import xarray as xr
import numpy as np
import pandas as pd
import cftime
from tqdm import tqdm
import cmocean as cm
import sys, os
from datetime import datetime

from dask.distributed import Client
import cosima_cookbook as cc
db = 'local_cc_scratch_Ryan_data.db'
session = cc.database.create_session(db)

In [45]:
# Load workers:
client = Client(threads_per_worker=1)
client

Perhaps you already have a cluster running?
Hosting the HTTP server on port 39239 instead
  warnings.warn(



Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/39239/status,
Dashboard: /proxy/39239/status,Workers: 28
Total threads: 28,Total memory: 0.98 TiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:37645,Workers: 28
Dashboard: /proxy/39239/status,Total threads: 28
Started: Just now,Total memory: 0.98 TiB
Comm: tcp://127.0.0.1:37155,Total threads: 1
Dashboard: /proxy/44953/status,Memory: 35.97 GiB
Nanny: tcp://127.0.0.1:38563,


## Load Online Heat Budget

In [46]:
#Winter MHW Coords!
# yt_coord = slice(-71,-68)
# xt_coord = slice(-190,-185)

#Weddell Case Study Coords!
yt_coord = slice(-70,-63)
xt_coord = slice(-60,20)

#Ross Case Study Coords!
# yt_coord = slice(-77, -70)
# xt_coord = slice(-200,-130)

time_coord = slice('2016-01-01','2023-12-31')

In [47]:
import xarray as xr
import glob

pattern = '/g/data/e14/ts7341/Figure_Code_For_Manuscript/Science/JRA_SST_Processed/Heat_Budget_Data/*.nc'
file_list = sorted(glob.glob(pattern))

ds = xr.open_mfdataset(file_list, combine='by_coords', engine='netcdf4')
ds = ds.sel(xt_ocean = xt_coord, yt_ocean = yt_coord).sel(time = slice('2016','2024'))



Found 43 files


In [48]:
data_sample = ds
data_sample

<xarray.Dataset> Size: 10GB
Dimensions:                 (xt_ocean: 320, yt_ocean: 66, time: 2914)
Coordinates:
  * xt_ocean                (xt_ocean) float64 3kB -59.88 -59.62 ... 19.62 19.88
  * yt_ocean                (yt_ocean) float64 528B -69.99 -69.88 ... -63.06
  * time                    (time) datetime64[ns] 23kB 2016-01-02 ... 2023-12-31
Data variables: (12/21)
    fixedh_tendency         (time, yt_ocean, xt_ocean) float64 492MB dask.array<chunksize=(365, 66, 320), meta=np.ndarray>
    temp_adv                (time, yt_ocean, xt_ocean) float64 492MB dask.array<chunksize=(365, 66, 320), meta=np.ndarray>
    submeso                 (time, yt_ocean, xt_ocean) float64 492MB dask.array<chunksize=(365, 66, 320), meta=np.ndarray>
    neut_diffusion          (time, yt_ocean, xt_ocean) float64 492MB dask.array<chunksize=(365, 66, 320), meta=np.ndarray>
    neutral_gm              (time, yt_ocean, xt_ocean) float64 492MB dask.array<chunksize=(365, 66, 320), meta=np.ndarray>
    temp_vdiffuse_k33       (time, yt_ocean, xt_ocean) float64 492MB dask.array<chunksize=(365, 66, 320), meta=np.ndarray>
    ...                      ...
    sens_heat               (time, yt_ocean, xt_ocean) float64 492MB dask.array<chunksize=(365, 66, 320), meta=np.ndarray>
    evap_heat               (time, yt_ocean, xt_ocean) float64 492MB dask.array<chunksize=(365, 66, 320), meta=np.ndarray>
    sw_heat_in_mld          (time, yt_ocean, xt_ocean) float64 492MB dask.array<chunksize=(365, 66, 320), meta=np.ndarray>
    residual                (time, yt_ocean, xt_ocean) float64 492MB dask.array<chunksize=(365, 66, 320), meta=np.ndarray>
    mlt_tendency            (time, yt_ocean, xt_ocean) float32 246MB dask.array<chunksize=(365, 66, 320), meta=np.ndarray>
    entrainment             (time, yt_ocean, xt_ocean) float64 492MB dask.array<chunksize=(365, 66, 320), meta=np.ndarray>
Attributes:
    Project:   Antarctic Seasonal Ice Zone MHWs 1982-2019 (plus extension to ...
    Author:    Tom Schmaltz - Hons Student 2025
    Data set:  ACCESS-OM2-025 IAF simulation forced by JRA55 (cycle6)
    Created:   2025-10-20
    Mail:      t.schmaltz@student.unsw.edu.au

In [49]:
## Add grouped terms to make analysis easier
data_sample['Grouped Advection'] = data_sample['temp_adv'] + \
                                   data_sample['submeso'] + \
                                   data_sample['neut_diffusion'] + \
                                   data_sample['neutral_gm'] + \
                                   data_sample['temp_vdiffuse_k33']

data_sample['Grouped Vertical Mixing'] = data_sample['temp_nonlocal_KPP'] + \
                                         data_sample['temp_vdiffuse_diff_cbt']

data_sample['Surface Fluxes Grouped'] = data_sample['temp_rivermix'] + \
                              data_sample['temp_vdiffuse_sbc'] + \
                              data_sample['frazil_3d'] + \
                              data_sample['sfc_hflux_pme']

data_sample['Surface Fluxes Less SW'] = data_sample['Surface Fluxes Grouped'] - \
                                        data_sample['swflx']

data_sample['mh_flux'] = data_sample['temp_vdiffuse_sbc'] - \
                              data_sample['lw_heat'] - \
                              data_sample['sens_heat'] - \
                              data_sample['evap_heat'] - \
                              data_sample['swflx']
                              # data_sample['frazil_3d']
                              # data_sample['sfc_hflux_pme'] - \
#                              data_sample['temp_rivermix']

data_sample['Shortwave Penetration'] = data_sample['sw_heat_in_mld']

# data_sample['Surface Fluxes Grouped less Shortwave & Mass Fluxes'] = data_sample['temp_rivermix'] + \
#                               data_sample['temp_vdiffuse_sbc'] + \
#                               data_sample['frazil_3d'] + \
#                               data_sample['sfc_hflux_pme'] - \
#                               data_sample['mh_flux'] - \
#                               data_sample['swflx']

data_sample['Net Shortwave'] = data_sample['swflx'] + data_sample['sw_heat_in_mld']

In [50]:
data_sample['time'] = data_sample['time'] - np.timedelta64(12, 'h') - np.timedelta64(1, 'D')

In [53]:
data_sample.load()

<xarray.Dataset> Size: 14GB
Dimensions:                  (xt_ocean: 320, yt_ocean: 66, time: 2914)
Coordinates:
  * xt_ocean                 (xt_ocean) float64 3kB -59.88 -59.62 ... 19.88
  * yt_ocean                 (yt_ocean) float64 528B -69.99 -69.88 ... -63.06
  * time                     (time) datetime64[ns] 23kB 2015-12-31T12:00:00 ....
Data variables: (12/28)
    fixedh_tendency          (time, yt_ocean, xt_ocean) float64 492MB 4.607e-...
    temp_adv                 (time, yt_ocean, xt_ocean) float64 492MB 2.579e-...
    submeso                  (time, yt_ocean, xt_ocean) float64 492MB 0.0 ......
    neut_diffusion           (time, yt_ocean, xt_ocean) float64 492MB 2.35e-0...
    neutral_gm               (time, yt_ocean, xt_ocean) float64 492MB -1.284e...
    temp_vdiffuse_k33        (time, yt_ocean, xt_ocean) float64 492MB 0.0 ......
    ...                       ...
    Grouped Vertical Mixing  (time, yt_ocean, xt_ocean) float64 492MB -1.433e...
    Surface Fluxes Grouped   (time, yt_ocean, xt_ocean) float64 492MB 5.424e-...
    Surface Fluxes Less SW   (time, yt_ocean, xt_ocean) float64 492MB -1.525e...
    mh_flux                  (time, yt_ocean, xt_ocean) float64 492MB -1.418e...
    Shortwave Penetration    (time, yt_ocean, xt_ocean) float64 492MB -1.992e...
    Net Shortwave            (time, yt_ocean, xt_ocean) float64 492MB 4.957e-...
Attributes:
    Project:   Antarctic Seasonal Ice Zone MHWs 1982-2019 (plus extension to ...
    Author:    Tom Schmaltz - Hons Student 2025
    Data set:  ACCESS-OM2-025 IAF simulation forced by JRA55 (cycle6)
    Created:   2025-10-20
    Mail:      t.schmaltz@student.unsw.edu.au

In [54]:
clim = xr.open_mfdataset('/scratch/e14/ts7341/Climatology_WS.nc').sel(xt_ocean = xt_coord, yt_ocean = yt_coord)

In [55]:
clim['Grouped Advection'] = clim['temp_adv'] + \
                                   clim['submeso'] + \
                                   clim['neut_diffusion'] + \
                                   clim['neutral_gm'] + \
                                   clim['temp_vdiffuse_k33']

clim['Grouped Vertical Mixing'] = clim['temp_nonlocal_KPP'] + \
                                         clim['temp_vdiffuse_diff_cbt']

clim['Surface Fluxes Grouped'] = clim['temp_rivermix'] + \
                              clim['temp_vdiffuse_sbc'] + \
                              clim['frazil_3d'] + \
                              clim['sfc_hflux_pme']

clim['mh_flux'] = clim['temp_vdiffuse_sbc'] - \
                              clim['lw_heat'] - \
                              clim['sens_heat'] - \
                              clim['evap_heat'] - \
                              clim['swflx'] 

# clim['Surface Fluxes Grouped less Shortwave & Mass Fluxes'] = clim['temp_rivermix'] + \
#                               clim['temp_vdiffuse_sbc'] + \
#                               clim['frazil_3d'] + \
#                               clim['sfc_hflux_pme'] - \
#                               clim['mh_flux'] - \
#                               clim['swflx']

clim['Surface Fluxes Less SW'] = clim['Surface Fluxes Grouped'] - \
                                        clim['swflx']

clim['Shortwave Penetration'] = clim['sw_heat_in_mld']

clim['Net Shortwave'] = clim['swflx'] + clim['sw_heat_in_mld']

In [56]:
clim

<xarray.Dataset> Size: 2GB
Dimensions:                  (xt_ocean: 320, yt_ocean: 66, dayofyear: 366)
Coordinates:
  * xt_ocean                 (xt_ocean) float64 3kB -59.88 -59.62 ... 19.88
  * yt_ocean                 (yt_ocean) float64 528B -69.99 -69.88 ... -63.06
  * dayofyear                (dayofyear) int64 3kB 1 2 3 4 5 ... 363 364 365 366
Data variables: (12/28)
    fixedh_tendency          (dayofyear, yt_ocean, xt_ocean) float64 62MB dask.array<chunksize=(23, 3, 25), meta=np.ndarray>
    temp_adv                 (dayofyear, yt_ocean, xt_ocean) float64 62MB dask.array<chunksize=(23, 3, 25), meta=np.ndarray>
    submeso                  (dayofyear, yt_ocean, xt_ocean) float64 62MB dask.array<chunksize=(23, 3, 25), meta=np.ndarray>
    neut_diffusion           (dayofyear, yt_ocean, xt_ocean) float64 62MB dask.array<chunksize=(23, 3, 25), meta=np.ndarray>
    neutral_gm               (dayofyear, yt_ocean, xt_ocean) float64 62MB dask.array<chunksize=(23, 3, 25), meta=np.ndarray>
    temp_vdiffuse_k33        (dayofyear, yt_ocean, xt_ocean) float64 62MB dask.array<chunksize=(23, 3, 25), meta=np.ndarray>
    ...                       ...
    Grouped Vertical Mixing  (dayofyear, yt_ocean, xt_ocean) float64 62MB dask.array<chunksize=(23, 3, 25), meta=np.ndarray>
    Surface Fluxes Grouped   (dayofyear, yt_ocean, xt_ocean) float64 62MB dask.array<chunksize=(23, 3, 25), meta=np.ndarray>
    Shortwave Penetration    (dayofyear, yt_ocean, xt_ocean) float64 62MB dask.array<chunksize=(23, 3, 25), meta=np.ndarray>
    mh_flux                  (dayofyear, yt_ocean, xt_ocean) float64 62MB dask.array<chunksize=(23, 3, 25), meta=np.ndarray>
    Surface Fluxes Less SW   (dayofyear, yt_ocean, xt_ocean) float64 62MB dask.array<chunksize=(23, 3, 25), meta=np.ndarray>
    Net Shortwave            (dayofyear, yt_ocean, xt_ocean) float64 62MB dask.array<chunksize=(23, 3, 25), meta=np.ndarray>

In [57]:
## Validate that this is correct here, it can cause headaches later
print("Data sample dayofyear range:",
      data_sample['time'].dt.dayofyear.min().item(),
      data_sample['time'].dt.dayofyear.max().item())

print("Clim dayofyear range:",
      clim['dayofyear'].min().item(),
      clim['dayofyear'].max().item())

Data sample dayofyear range: 1 366
Clim dayofyear range: 1 366


In [59]:
clim

<xarray.Dataset> Size: 2GB
Dimensions:                  (xt_ocean: 320, yt_ocean: 66, dayofyear: 366)
Coordinates:
  * xt_ocean                 (xt_ocean) float64 3kB -59.88 -59.62 ... 19.88
  * yt_ocean                 (yt_ocean) float64 528B -69.99 -69.88 ... -63.06
  * dayofyear                (dayofyear) int64 3kB 1 2 3 4 5 ... 363 364 365 366
Data variables: (12/28)
    fixedh_tendency          (dayofyear, yt_ocean, xt_ocean) float64 62MB dask.array<chunksize=(23, 3, 25), meta=np.ndarray>
    temp_adv                 (dayofyear, yt_ocean, xt_ocean) float64 62MB dask.array<chunksize=(23, 3, 25), meta=np.ndarray>
    submeso                  (dayofyear, yt_ocean, xt_ocean) float64 62MB dask.array<chunksize=(23, 3, 25), meta=np.ndarray>
    neut_diffusion           (dayofyear, yt_ocean, xt_ocean) float64 62MB dask.array<chunksize=(23, 3, 25), meta=np.ndarray>
    neutral_gm               (dayofyear, yt_ocean, xt_ocean) float64 62MB dask.array<chunksize=(23, 3, 25), meta=np.ndarray>
    temp_vdiffuse_k33        (dayofyear, yt_ocean, xt_ocean) float64 62MB dask.array<chunksize=(23, 3, 25), meta=np.ndarray>
    ...                       ...
    Grouped Vertical Mixing  (dayofyear, yt_ocean, xt_ocean) float64 62MB dask.array<chunksize=(23, 3, 25), meta=np.ndarray>
    Surface Fluxes Grouped   (dayofyear, yt_ocean, xt_ocean) float64 62MB dask.array<chunksize=(23, 3, 25), meta=np.ndarray>
    Shortwave Penetration    (dayofyear, yt_ocean, xt_ocean) float64 62MB dask.array<chunksize=(23, 3, 25), meta=np.ndarray>
    mh_flux                  (dayofyear, yt_ocean, xt_ocean) float64 62MB dask.array<chunksize=(23, 3, 25), meta=np.ndarray>
    Surface Fluxes Less SW   (dayofyear, yt_ocean, xt_ocean) float64 62MB dask.array<chunksize=(23, 3, 25), meta=np.ndarray>
    Net Shortwave            (dayofyear, yt_ocean, xt_ocean) float64 62MB dask.array<chunksize=(23, 3, 25), meta=np.ndarray>

In [60]:
clim_daily = clim.sel(dayofyear=data_sample['time'].dt.dayofyear)
clim_daily = clim_daily.assign_coords(time=data_sample['time'])
clim_daily = clim_daily.sel(yt_ocean = yt_coord, xt_ocean = xt_coord)

In [61]:
clim_daily

<xarray.Dataset> Size: 14GB
Dimensions:                  (xt_ocean: 320, yt_ocean: 66, time: 2914)
Coordinates:
  * xt_ocean                 (xt_ocean) float64 3kB -59.88 -59.62 ... 19.88
  * yt_ocean                 (yt_ocean) float64 528B -69.99 -69.88 ... -63.06
    dayofyear                (time) int64 23kB 365 1 2 3 4 ... 360 361 362 363
  * time                     (time) datetime64[ns] 23kB 2015-12-31T12:00:00 ....
Data variables: (12/28)
    fixedh_tendency          (time, yt_ocean, xt_ocean) float64 492MB dask.array<chunksize=(22, 3, 25), meta=np.ndarray>
    temp_adv                 (time, yt_ocean, xt_ocean) float64 492MB dask.array<chunksize=(22, 3, 25), meta=np.ndarray>
    submeso                  (time, yt_ocean, xt_ocean) float64 492MB dask.array<chunksize=(22, 3, 25), meta=np.ndarray>
    neut_diffusion           (time, yt_ocean, xt_ocean) float64 492MB dask.array<chunksize=(22, 3, 25), meta=np.ndarray>
    neutral_gm               (time, yt_ocean, xt_ocean) float64 492MB dask.array<chunksize=(22, 3, 25), meta=np.ndarray>
    temp_vdiffuse_k33        (time, yt_ocean, xt_ocean) float64 492MB dask.array<chunksize=(22, 3, 25), meta=np.ndarray>
    ...                       ...
    Grouped Vertical Mixing  (time, yt_ocean, xt_ocean) float64 492MB dask.array<chunksize=(22, 3, 25), meta=np.ndarray>
    Surface Fluxes Grouped   (time, yt_ocean, xt_ocean) float64 492MB dask.array<chunksize=(22, 3, 25), meta=np.ndarray>
    Shortwave Penetration    (time, yt_ocean, xt_ocean) float64 492MB dask.array<chunksize=(22, 3, 25), meta=np.ndarray>
    mh_flux                  (time, yt_ocean, xt_ocean) float64 492MB dask.array<chunksize=(22, 3, 25), meta=np.ndarray>
    Surface Fluxes Less SW   (time, yt_ocean, xt_ocean) float64 492MB dask.array<chunksize=(22, 3, 25), meta=np.ndarray>
    Net Shortwave            (time, yt_ocean, xt_ocean) float64 492MB dask.array<chunksize=(22, 3, 25), meta=np.ndarray>

In [62]:
## Ensure that you have the anomaly!
anom = data_sample - clim_daily
anom.load()

This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the graph without repetition.
See also https://docs.dask.org/en/stable/best-practices.html#load-data-with-dask for more information.
  warnings.warn(



<xarray.Dataset> Size: 14GB
Dimensions:                  (xt_ocean: 320, yt_ocean: 66, time: 2914)
Coordinates:
  * xt_ocean                 (xt_ocean) float64 3kB -59.88 -59.62 ... 19.88
  * yt_ocean                 (yt_ocean) float64 528B -69.99 -69.88 ... -63.06
  * time                     (time) datetime64[ns] 23kB 2015-12-31T12:00:00 ....
    dayofyear                (time) int64 23kB 365 1 2 3 4 ... 360 361 362 363
Data variables: (12/28)
    fixedh_tendency          (time, yt_ocean, xt_ocean) float64 492MB 4.5e-06...
    temp_adv                 (time, yt_ocean, xt_ocean) float64 492MB 2.566e-...
    submeso                  (time, yt_ocean, xt_ocean) float64 492MB 1.367e-...
    neut_diffusion           (time, yt_ocean, xt_ocean) float64 492MB -1.586e...
    neutral_gm               (time, yt_ocean, xt_ocean) float64 492MB -1.114e...
    temp_vdiffuse_k33        (time, yt_ocean, xt_ocean) float64 492MB -2.199e...
    ...                       ...
    Grouped Vertical Mixing  (time, yt_ocean, xt_ocean) float64 492MB -1.437e...
    Surface Fluxes Grouped   (time, yt_ocean, xt_ocean) float64 492MB 4.706e-...
    Surface Fluxes Less SW   (time, yt_ocean, xt_ocean) float64 492MB -1.076e...
    mh_flux                  (time, yt_ocean, xt_ocean) float64 492MB -5.638e...
    Shortwave Penetration    (time, yt_ocean, xt_ocean) float64 492MB -1.323e...
    Net Shortwave            (time, yt_ocean, xt_ocean) float64 492MB 3.491e-...

## Load Offline Terms – Need to do for Reynolds Decomposition

In [63]:
import cosima_cookbook as cc
db = 'local_cc_scratch_Ryan_data.db'
session = cc.database.create_session(db)

time = time_coord
# yt_ocean = yt_coord
# xt_ocean = xt_coord 

clim_time = slice('1982-01-01','2011-12-31')

In [64]:
net_sfc_heating = cc.querying.getvar(
    expt='025deg_jra55_iaf_cycle6_online_mlt',
    variable='net_sfc_heating',
    session=session,
    frequency='1 daily',
    attrs={'cell_methods': 'time: mean'},
    chunks={'time': -1, 'xt_ocean': '400MB', 'yt_ocean': '400MB'}
).sel(time = time).\
sel(xt_ocean = xt_coord, yt_ocean = yt_coord)

hflx_sample = net_sfc_heating

In [65]:
net_sfc_heating_clim = cc.querying.getvar(
    expt='025deg_jra55_iaf_cycle6_online_mlt',
    variable='net_sfc_heating',
    session=session,
    frequency='1 daily',
    attrs={'cell_methods': 'time: mean'},
    chunks={'time': -1, 'xt_ocean': '400MB', 'yt_ocean': '400MB'}
).sel(time = clim_time).\
sel(xt_ocean = xt_coord, yt_ocean = yt_coord)

hflx_clim_sample = net_sfc_heating

In [66]:
def smoothedClima_mhw(ds):
    """
    Replicate the climatology calculation used in the marineHeatWave.py algorithm.
    Input:
        - ds: xr.Dataset containing (at least) `time` (as np.datetime64 format) and any variable over 
              which the climatology should be calculated (e.g. SST, ice concentration, etc.)
    Output:
        - SmoothThresh: a xr.Dataset of same size as ds, except for the `time` dimension that became a 
        `dayofyear` dimension of size 366. This dataset contains
        
    Note that contrarily to the marineheatwave algorithm, the 366th day is not inserted on 
    the 29th of February via interpolation but is calculated as the other days and is inserted on
    the 31st of December. This subtlety is arising from the handling of dayofyear by xarray (or rather pandas,
    used internally). On leap years, 31st of December is doy 366, while on non-leap years, it is doy 365.
    """
    if ~np.isin("time",ds.dims):
        print("No 'time' dimensions in the dataset")
        return
    # Calculate a rolling mean first, with a 11 days window. 
    ds_rolled = ds.rolling(time=11, min_periods=1, center=True).mean()
    # Calculate the mean over dayofyear
    ds_clim_doy = ds_rolled.groupby("time.dayofyear").mean()
    # If I just smooth over that, my beginning and end will be weird.
    # A solution is to replicate 3 times, do the running average, and get the middle year.
    stackedClim = xr.concat([ds_clim_doy,ds_clim_doy,ds_clim_doy],dim='year').stack(time={'year','dayofyear'})
    # Smooth the time series
    smoothedClim = stackedClim.rolling(time=31,
                                       min_periods=1,
                                       center=True).mean()
    # Extract the middle year and rearrange the dimensions to only keep "dayofyear"
    tmpSmooClim = smoothedClim.where(smoothedClim.year==1,drop=True).drop('year')
    ds_smoothedClim = tmpSmooClim.rename({'time':'dayofyear'}).assign_coords(dayofyear=ds_clim_doy.dayofyear.data)
    return ds_smoothedClim

In [67]:
hflx_clim = smoothedClima_mhw(hflx_clim_sample) 

  tmpSmooClim = smoothedClim.where(smoothedClim.year==1,drop=True).drop('year')



In [68]:
hflx_clim_daily = hflx_clim.sel(dayofyear=hflx_sample['time'].dt.dayofyear)
hflx_clim_daily = hflx_clim_daily.assign_coords(time=hflx_sample['time'])
hflx_clim_daily.compute()

<xarray.DataArray 'net_sfc_heating' (yt_ocean: 66, xt_ocean: 320, time: 2920)> Size: 247MB
array([[[  3.9326644,   3.806252 ,   3.7234194, ...,   5.2159524,
           4.919163 ,   4.647498 ],
        [  3.6210232,   3.5509112,   3.5211215, ...,   4.593429 ,
           4.3545737,   4.1331162],
        [  4.2071137,   4.1602173,   4.1437945, ...,   4.8484416,
           4.6826277,   4.5305853],
        ...,
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan]],

       [[  2.8176389,   2.6779287,   2.595911 , ...,   4.3879614,
           4.0199757,   3.6855412],
        [  3.0298803,   2.9681866,   2.9547393, ...,   4.0797915,
           3.813602 ,   3.57086  ],
        [  3.9650283,   3.9346366,   3.937126 , ...,   4.5226035,
           4.3659854,   4.2263417],
...
        [ 90.45438  ,  88.97311  ,  87.526855 , ...,  98.19476  ,
          97.29501  ,  96.19382  ],
        [ 90.87034  ,  89.36354  ,  87.89557  , ...,  98.73221  ,
          97.827835 ,  96.717445 ],
        [ 91.03816  ,  89.53067  ,  88.06102  , ...,  98.83376  ,
          97.95143  ,  96.86232  ]],

       [[121.86588  , 120.510414 , 118.90913  , ..., 129.88478  ,
         128.42618  , 126.92188  ],
        [123.10701  , 121.71712  , 120.07115  , ..., 131.33167  ,
         129.83194  , 128.28645  ],
        [123.024185 , 121.61254  , 119.935356 , ..., 131.43582  ,
         129.89783  , 128.31282  ],
        ...,
        [ 90.91065  ,  89.4278   ,  87.98426  , ...,  98.847336 ,
          97.89212  ,  96.7445   ],
        [ 91.346725 ,  89.83556  ,  88.369026 , ...,  99.44562  ,
          98.477135 ,  97.312706 ],
        [ 91.53861  ,  90.0207   ,  88.54823  , ...,  99.630104 ,
          98.671524 ,  97.51766  ]]], dtype=float32)
Coordinates:
  * xt_ocean   (xt_ocean) float64 3kB -59.88 -59.62 -59.38 ... 19.38 19.62 19.88
  * yt_ocean   (yt_ocean) float64 528B -69.99 -69.88 -69.78 ... -63.17 -63.06
    dayofyear  (time) int64 23kB 1 2 3 4 5 6 7 8 ... 357 358 359 360 361 362 363
  * time       (time) datetime64[ns] 23kB 2016-01-01T12:00:00 ... 2023-12-29T...
Attributes:
    long_name:      surface ocean heat flux coming through coupler and mass t...
    units:          Watts/m^2
    valid_range:    [-10000.  10000.]
    cell_methods:   time: mean
    time_avg_info:  average_T1,average_T2,average_DT
    ncfiles:        ['/g/data/e14/rmh561/access-om2/archive/025deg_jra55_iaf_...
    contact:        Ryan Holmes
    email:          ryan.holmes@unsw.edu.au
    created:        2020-11-03
    description:    0.25 degree ACCESS-OM2 global model configuration under i...

In [69]:
mld_sample = cc.querying.getvar(expt='025deg_jra55_iaf_cycle6_online_mlt', variable='mld', 
                          session=session, frequency='1 daily',
                          attrs={'cell_methods': 'time: mean'},
                          chunks={'time': -1, 'xt_ocean': '400MB', 'yt_ocean': '400MB'}).\
                          sel(time = time_coord).\
                          sel(xt_ocean = xt_coord, yt_ocean = yt_coord)
mld_sample.compute()

<xarray.DataArray 'mld' (time: 2920, yt_ocean: 66, xt_ocean: 320)> Size: 247MB
array([[[ 5.2831645,  4.8732276,  4.71301  , ...,        nan,
                nan,        nan],
        [ 5.3806057,  4.8209953,  4.6718106, ...,  3.763672 ,
          3.6903431,  3.6429198],
        [ 5.385478 ,  4.7519927,  4.5587544, ...,  3.742304 ,
          3.6704385,  3.6448355],
        ...,
        [ 7.2889433,  7.525671 ,  7.657929 , ...,  9.311692 ,
          9.38454  ,  9.465901 ],
        [ 7.13598  ,  7.0428624,  7.2833567, ...,  9.220128 ,
          9.334324 ,  9.415399 ],
        [ 7.7492747,  7.2228975,  7.242163 , ...,  9.337699 ,
          9.350188 ,  9.412458 ]],

       [[ 8.9679785,  8.837684 ,  8.661904 , ...,        nan,
                nan,        nan],
        [ 8.456398 ,  8.286562 ,  8.274271 , ...,  3.6452656,
          3.6449752,  3.6429048],
        [ 7.842669 ,  7.8826337,  8.04154  , ...,  3.6463103,
          3.6462247,  3.644829 ],
...
        [13.471653 , 13.516129 , 14.423725 , ..., 14.828419 ,
         14.516388 , 14.049643 ],
        [15.4739485, 15.729838 , 16.2434   , ..., 15.261224 ,
         14.949551 , 14.968198 ],
        [16.006628 , 16.490408 , 16.709902 , ..., 15.510455 ,
         15.404646 , 15.375141 ]],

       [[ 7.192521 ,  9.15587  ,  9.549715 , ...,        nan,
                nan,        nan],
        [ 8.733296 ,  9.575486 ,  9.823123 , ...,  3.6452167,
          3.6449187,  3.6428163],
        [ 8.99873  ,  9.660477 ,  9.796691 , ...,  3.6462893,
          3.6462018,  3.644779 ],
        ...,
        [13.117651 , 13.045295 , 12.985387 , ..., 19.848345 ,
         19.430866 , 19.189577 ],
        [16.193626 , 15.385113 , 14.833255 , ..., 20.254887 ,
         19.79114  , 19.64448  ],
        [17.065271 , 16.79645  , 15.820346 , ..., 20.736572 ,
         20.128807 , 19.960835 ]]], dtype=float32)
Coordinates:
  * xt_ocean  (xt_ocean) float64 3kB -59.88 -59.62 -59.38 ... 19.38 19.62 19.88
  * yt_ocean  (yt_ocean) float64 528B -69.99 -69.88 -69.78 ... -63.17 -63.06
  * time      (time) datetime64[ns] 23kB 2016-01-01T12:00:00 ... 2023-12-29T1...
Attributes:
    long_name:      mixed layer depth determined by density criteria
    units:          m
    valid_range:    [      0. 1000000.]
    cell_methods:   time: mean
    time_avg_info:  average_T1,average_T2,average_DT
    standard_name:  ocean_mixed_layer_thickness_defined_by_sigma_t
    ncfiles:        ['/g/data/e14/rmh561/access-om2/archive/025deg_jra55_iaf_...
    contact:        Ryan Holmes
    email:          ryan.holmes@unsw.edu.au
    created:        2020-11-03
    description:    0.25 degree ACCESS-OM2 global model configuration under i...

In [70]:
mld_clim_sample = cc.querying.getvar(expt='025deg_jra55_iaf_cycle6_online_mlt', variable='mld', 
                          session=session, frequency='1 daily',
                          attrs={'cell_methods': 'time: mean'},
                          chunks={'time': -1, 'xt_ocean': '400MB', 'yt_ocean': '400MB'}).\
                          sel(time = clim_time).\
                          sel(xt_ocean = xt_coord, yt_ocean = yt_coord)

mld_clim_sample = mld_clim_sample.load()

In [71]:
mld_clim = smoothedClima_mhw(mld_clim_sample)

  tmpSmooClim = smoothedClim.where(smoothedClim.year==1,drop=True).drop('year')



In [72]:
mld_clim_daily = mld_clim.sel(dayofyear=mld_sample['time'].dt.dayofyear)
mld_clim_daily = mld_clim_daily.assign_coords(time=mld_sample['time'])
mld_clim_daily.load()

<xarray.DataArray 'mld' (yt_ocean: 66, xt_ocean: 320, time: 2920)> Size: 247MB
array([[[ 8.188327 ,  8.170337 ,  8.166637 , ...,  8.873421 ,
          8.670442 ,  8.517678 ],
        [ 8.575113 ,  8.53742  ,  8.514259 , ...,  9.379518 ,
          9.152526 ,  8.978341 ],
        [ 8.801009 ,  8.752012 ,  8.7179575, ...,  9.664926 ,
          9.427044 ,  9.244065 ],
        ...,
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan]],

       [[ 8.290396 ,  8.2716675,  8.266931 , ...,  9.012434 ,
          8.793096 ,  8.630793 ],
        [ 8.708302 ,  8.674886 ,  8.65655  , ...,  9.570802 ,
          9.316151 ,  9.121538 ],
        [ 8.90476  ,  8.855335 ,  8.821522 , ...,  9.850218 ,
          9.581639 ,  9.374481 ],
...
        [12.038177 , 12.136057 , 12.241034 , ..., 11.705255 ,
         11.719961 , 11.751936 ],
        [12.021197 , 12.118041 , 12.222085 , ..., 11.6920805,
         11.706615 , 11.737933 ],
        [12.05765  , 12.157412 , 12.264907 , ..., 11.71621  ,
         11.733489 , 11.766777 ]],

       [[17.979776 , 17.939451 , 17.91285  , ..., 18.557034 ,
         18.415884 , 18.294472 ],
        [17.916649 , 17.884644 , 17.867579 , ..., 18.457956 ,
         18.319777 , 18.202042 ],
        [18.033085 , 18.010532 , 18.003862 , ..., 18.509119 ,
         18.381586 , 18.274916 ],
        ...,
        [12.168161 , 12.267038 , 12.37305  , ..., 11.829907 ,
         11.845014 , 11.877276 ],
        [12.129319 , 12.226254 , 12.330451 , ..., 11.799505 ,
         11.813927 , 11.845003 ],
        [12.108491 , 12.206098 , 12.311273 , ..., 11.7787695,
         11.793566 , 11.824235 ]]], dtype=float32)
Coordinates:
  * xt_ocean   (xt_ocean) float64 3kB -59.88 -59.62 -59.38 ... 19.38 19.62 19.88
  * yt_ocean   (yt_ocean) float64 528B -69.99 -69.88 -69.78 ... -63.17 -63.06
    dayofyear  (time) int64 23kB 1 2 3 4 5 6 7 8 ... 357 358 359 360 361 362 363
  * time       (time) datetime64[ns] 23kB 2016-01-01T12:00:00 ... 2023-12-29T...
Attributes:
    long_name:      mixed layer depth determined by density criteria
    units:          m
    valid_range:    [      0. 1000000.]
    cell_methods:   time: mean
    time_avg_info:  average_T1,average_T2,average_DT
    standard_name:  ocean_mixed_layer_thickness_defined_by_sigma_t
    ncfiles:        ['/g/data/e14/rmh561/access-om2/archive/025deg_jra55_iaf_...
    contact:        Ryan Holmes
    email:          ryan.holmes@unsw.edu.au
    created:        2020-11-03
    description:    0.25 degree ACCESS-OM2 global model configuration under i...

In [73]:
# Reynolds Decomposition 
# In our study we don't use any of the correction terms – it is so close e-8 to being closed anyway and these terms are less than the accuracy loss from going offline by ORDERS OF MAGNITUDE
%matplotlib inline
rho0 = 1035.
Cp = 3992.10322329649

def reynolds_decompose(Q_2023, Q_clim, H_2023, H_clim):
    """Apply Reynolds decomposition to heat terms – Implemented as we Derived in Appendix D, also consistent with Richaud et al. (2024) study implementation"""
    Q_anom = Q_2023 - Q_clim
    invH_2023 = 1 / H_2023
    invH_clim = 1 / H_clim
    invH_anom = invH_2023 - invH_clim

    # Decomposition terms
    QoverH = Q_2023 / H_2023
    Qbar_1ovrHbar = Q_clim / H_clim
    Qprime_1ovrHbar = Q_anom * invH_clim
    Qbar_1ovrHprime = Q_clim * invH_anom
    Qprime_1ovrHprime = Q_anom * invH_anom

    Residual = QoverH - (Qbar_1ovrHbar + Qprime_1ovrHbar + Qbar_1ovrHprime + Qprime_1ovrHprime)
    
    # Time mean of covariant term
    Qprime_1ovrHprime_mean = np.mean(Qprime_1ovrHprime, axis=0)

    # Corrected climatological Qsurf 
    Qbar_corrected = Qbar_1ovrHbar + Qprime_1ovrHprime_mean

    #ANOMALOUS ANOMALY
    Qprime_1ovrHprime_anom = Qprime_1ovrHprime - Qprime_1ovrHprime.mean(dim='time')

    Residual_corrected = QoverH - (Qbar_corrected + Qprime_1ovrHbar + 
                                   Qbar_1ovrHprime + Qprime_1ovrHprime_anom)

    return {
        "QoverH": QoverH,
        "Qbar_1ovrHbar": Qbar_1ovrHbar,
        "Qprime_1ovrHbar": Qprime_1ovrHbar,
        "Qbar_1ovrHprime": Qbar_1ovrHprime,
        "Qprime_1ovrHprime": Qprime_1ovrHprime,
        "Qprime_1ovrHprime_mean": Qprime_1ovrHprime_mean,
        "Qbar_corrected": Qbar_corrected,
        "Qprime_1ovrHprime_anom": Qprime_1ovrHprime_anom,
        "Residual": Residual,
        "Residual_Corrected": Residual_corrected
    }

Q_2023 = hflx_sample
Q_clim = hflx_clim_daily

H_2023 = mld_sample
H_clim = mld_clim_daily

results = reynolds_decompose(Q_2023, Q_clim, H_2023, H_clim)

ds_reynolds = xr.Dataset({
    "QoverH": results["QoverH"],
    "Qbar_1ovrHbar": results["Qbar_1ovrHbar"],
    "Qprime_1ovrHbar": results["Qprime_1ovrHbar"],
    "Qbar_1ovrHprime": results["Qbar_1ovrHprime"],
    "Qprime_1ovrHprime": results["Qprime_1ovrHprime"],
    "Residual": results["Residual"],
    "Residual_Corrected": results["Residual_Corrected"],
})

ds_reynolds = ds_reynolds/rho0/Cp*3600*24
ds_reynolds = ds_reynolds.sel(time = time)
ds_reynolds.compute()

This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the graph without repetition.
See also https://docs.dask.org/en/stable/best-practices.html#load-data-with-dask for more information.
  warnings.warn(



<xarray.Dataset> Size: 2GB
Dimensions:             (xt_ocean: 320, yt_ocean: 66, time: 2920)
Coordinates:
  * xt_ocean            (xt_ocean) float64 3kB -59.88 -59.62 ... 19.62 19.88
  * yt_ocean            (yt_ocean) float64 528B -69.99 -69.88 ... -63.17 -63.06
  * time                (time) datetime64[ns] 23kB 2016-01-01T12:00:00 ... 20...
    dayofyear           (time) int64 23kB 1 2 3 4 5 6 ... 359 360 361 362 363
Data variables:
    QoverH              (time, yt_ocean, xt_ocean) float32 247MB 0.1304 ... 0...
    Qbar_1ovrHbar       (yt_ocean, xt_ocean, time) float32 247MB 0.01004 ... ...
    Qprime_1ovrHbar     (time, yt_ocean, xt_ocean) float32 247MB 0.07408 ... ...
    Qbar_1ovrHprime     (yt_ocean, xt_ocean, time) float32 247MB 0.005523 ......
    Qprime_1ovrHprime   (time, yt_ocean, xt_ocean) float32 247MB 0.04073 ... ...
    Residual            (time, yt_ocean, xt_ocean) float32 247MB 9.971e-09 .....
    Residual_Corrected  (time, yt_ocean, xt_ocean) float32 247MB 0.0 ... -9.9...

## Integrate Terms

### Case Study 1: Weddell Sea

In [74]:
ds = xr.open_dataset('/scratch/e14/ts7341/SEANOE/MHW_Statistics_Mask/full_MHW_data_1982_2024_WS.nc')

In [75]:
ds = ds.transpose('lat', 'lon', 'event')
ds = ds.sel(lat = yt_coord, lon = xt_coord)
ds

<xarray.Dataset> Size: 96MB
Dimensions:          (lat: 66, lon: 320, event: 57)
Coordinates:
  * lon              (lon) float64 3kB -59.88 -59.62 -59.38 ... 19.62 19.88
  * lat              (lat) float64 528B -69.99 -69.88 -69.78 ... -63.17 -63.06
  * event            (event) int64 456B 0 1 2 3 4 5 6 7 ... 50 51 52 53 54 55 56
Data variables:
    index_start      (lat, lon, event) float64 10MB ...
    index_end        (lat, lon, event) float64 10MB ...
    date_start       (lat, lon, event) datetime64[ns] 10MB ...
    date_end         (lat, lon, event) datetime64[ns] 10MB ...
    duration         (lat, lon, event) float64 10MB ...
    n_event          (lat, lon) int64 169kB ...
    index_peak       (lat, lon, event) float64 10MB ...
    date_peak        (lat, lon, event) datetime64[ns] 10MB ...
    intensity_max    (lat, lon, event) float64 10MB ...
    intensity_mean   (lat, lon, event) float64 10MB ...
    intensity_cumul  (lat, lon, event) float64 10MB ...

In [76]:
ds

<xarray.Dataset> Size: 96MB
Dimensions:          (lat: 66, lon: 320, event: 57)
Coordinates:
  * lon              (lon) float64 3kB -59.88 -59.62 -59.38 ... 19.62 19.88
  * lat              (lat) float64 528B -69.99 -69.88 -69.78 ... -63.17 -63.06
  * event            (event) int64 456B 0 1 2 3 4 5 6 7 ... 50 51 52 53 54 55 56
Data variables:
    index_start      (lat, lon, event) float64 10MB ...
    index_end        (lat, lon, event) float64 10MB ...
    date_start       (lat, lon, event) datetime64[ns] 10MB ...
    date_end         (lat, lon, event) datetime64[ns] 10MB ...
    duration         (lat, lon, event) float64 10MB ...
    n_event          (lat, lon) int64 169kB ...
    index_peak       (lat, lon, event) float64 10MB ...
    date_peak        (lat, lon, event) datetime64[ns] 10MB ...
    intensity_max    (lat, lon, event) float64 10MB ...
    intensity_mean   (lat, lon, event) float64 10MB ...
    intensity_cumul  (lat, lon, event) float64 10MB ...

In [77]:
## If you want to filter the dataset from a year, you should do it here! It is a bit clunky with the way it has to be filtered!
# condition = ds['date_start'].dt.year >= 2016
# ds_filtered = ds.where(condition, drop=True)

# new_event_count = ds_filtered['index_start'].count(dim='event')

# ds_filtered['n_event'] = new_event_count

# print(ds_filtered)
# ds = ds_filtered

Filtering dataset for year >= 2014...
<xarray.Dataset> Size: 90MB
Dimensions:          (lat: 66, lon: 320, event: 53)
Coordinates:
  * lon              (lon) float64 3kB -59.88 -59.62 -59.38 ... 19.62 19.88
  * lat              (lat) float64 528B -69.99 -69.88 -69.78 ... -63.17 -63.06
  * event            (event) int64 424B 4 5 6 7 8 9 10 ... 50 51 52 53 54 55 56
Data variables:
    index_start      (lat, lon, event) float64 9MB nan nan nan ... nan nan nan
    index_end        (lat, lon, event) float64 9MB nan nan nan ... nan nan nan
    date_start       (lat, lon, event) datetime64[ns] 9MB NaT NaT ... NaT NaT
    date_end         (lat, lon, event) datetime64[ns] 9MB NaT NaT ... NaT NaT
    duration         (lat, lon, event) float64 9MB nan nan nan ... nan nan nan
    n_event          (lat, lon) int64 169kB 2 2 2 2 1 1 1 1 ... 7 7 7 7 7 8 8 8
    index_peak       (lat, lon, event) float64 9MB nan nan nan ... nan nan nan
    date_peak        (lat, lon, event) datetime64[ns] 9MB NaT NaT 

In [79]:
print(anom)
print(ds_reynolds)
print(ds)

## I am not sure why this is the case, but you have to reorder the variables here because it causes the coordinates to be out of order 
## It has to do with the way the Reynolds decomposition is done, I think
ds_reynolds['Qbar_1ovrHprime'] = ds_reynolds['Qbar_1ovrHprime'].transpose('time', 'yt_ocean', 'xt_ocean')

<xarray.Dataset> Size: 14GB
Dimensions:                  (xt_ocean: 320, yt_ocean: 66, time: 2914)
Coordinates:
  * xt_ocean                 (xt_ocean) float64 3kB -59.88 -59.62 ... 19.88
  * yt_ocean                 (yt_ocean) float64 528B -69.99 -69.88 ... -63.06
  * time                     (time) datetime64[ns] 23kB 2015-12-31T12:00:00 ....
    dayofyear                (time) int64 23kB 365 1 2 3 4 ... 360 361 362 363
Data variables: (12/28)
    fixedh_tendency          (time, yt_ocean, xt_ocean) float64 492MB 4.5e-06...
    temp_adv                 (time, yt_ocean, xt_ocean) float64 492MB 2.566e-...
    submeso                  (time, yt_ocean, xt_ocean) float64 492MB 1.367e-...
    neut_diffusion           (time, yt_ocean, xt_ocean) float64 492MB -1.586e...
    neutral_gm               (time, yt_ocean, xt_ocean) float64 492MB -1.114e...
    temp_vdiffuse_k33        (time, yt_ocean, xt_ocean) float64 492MB -2.199e...
    ...                       ...
    Grouped Vertical Mixing  (

In [80]:
anom = anom.rename({'xt_ocean':'lon','yt_ocean':'lat'})

In [81]:
##Need to do the chunk -1 to get it along one coordinate
anom = anom.chunk({'time':-1})

In [82]:
ds_reynolds = ds_reynolds.rename({'xt_ocean':'lon','yt_ocean':'lat'})

In [83]:
##Need to do the chunk -1 to get it along one coordinate
ds_reynolds = ds_reynolds.chunk({'time':-1})

In [ ]:
anom = anom.sel(time = slice('1982-01-01','2023-12-29'))

### Fig A5c,f – Create the melt in the lead-up to the event dataset

In [28]:
## Melt
import numpy as np
import xarray as xr
from tqdm.auto import tqdm
from itertools import product

## just melt flux here, so make sure it is loaded in properly
mh_flux_anom = anom * 3600 * 24
mh_flux_anom = mh_flux_anom.rename({'yt_ocean':'lat', 'xt_ocean':'lon'})

time_anom_numpy = anom['time'].values

def calculate_precursor_mh_flux_sum(date_starts, mh_flux_ts_vals, time_coords_anom):
    """
    For a single grid point, calculates the sum of mh_flux anomaly
    in the 3 days leading up to the start of each event.
    """
    nevents = date_starts.shape[0]
   
    mh_flux_sum_out = np.full(nevents, np.nan, dtype=float)

    for k in range(nevents):
        event_start = date_starts[k]
        # Skip if the event start date is not valid
        if np.isnat(event_start):
            continue

        # Get the lead up period I defined in the paper
        precursor_start = event_start - np.timedelta64(3, 'D')

        # Create a mask for this period
        mask_anom = (time_coords_anom >= precursor_start) & (time_coords_anom <= event_start)

        # Calculate the sum for the precursor period using the mask
        # np.nansum is used to safely handle any potential missing data points
        mh_flux_sum_out[k] = np.nansum(mh_flux_ts_vals[mask_anom])

    return mh_flux_sum_out

# Need to loop through the Weddell Sea in tiles because it's huge
tile_size_lat = 40
tile_size_lon = 80
list_of_processed_tiles = []

lat_starts = range(0, ds.sizes['lat'], tile_size_lat)
lon_starts = range(0, ds.sizes['lon'], tile_size_lon)
tile_combinations = list(product(lat_starts, lon_starts))

for i_start, j_start in tqdm(tile_combinations, desc="Processing Tiles"):
    # Define the slices for the current tile
    lat_slice = slice(i_start, i_start + tile_size_lat)
    lon_slice = slice(j_start, j_start + tile_size_lon)

    mh_flux_tile_lazy = mh_flux_anom.isel(lat=lat_slice, lon=lon_slice)
    ds_tile_lazy = ds.isel(lat=lat_slice, lon=lon_slice)

    # Load the data for the tile into memory
    ds_tile = ds_tile_lazy.load()
    mh_flux_tile = mh_flux_tile_lazy.load()

    tile_nlat, tile_nlon = ds_tile.sizes['lat'], ds_tile.sizes['lon']

    # Np array to store the results for this tile
    tile_mh_flux_sum = np.full((tile_nlat, tile_nlon, ds.sizes['event']), np.nan, dtype=float)

    # Iterate over each grid point 
    for i_tile in range(tile_nlat):
        for j_tile in range(tile_nlon):
            # Extract the time series and event dates for the specific point
            date_starts_point = ds_tile['date_start'].values[i_tile, j_tile, :]
            mh_flux_ts_point = mh_flux_tile.values[:, i_tile, j_tile]

            results = calculate_precursor_mh_flux_sum(
                date_starts_point, mh_flux_ts_point, time_anom_numpy
            )

            # Store the results in the tile's output array
            tile_mh_flux_sum[i_tile, j_tile, :] = results

    # Create an xarray Dataset for the processed tile
    tile_result_ds = xr.Dataset(
        data_vars={
            'precursor_mh_flux_sum': (('lat', 'lon', 'event'), tile_mh_flux_sum),
        },
        coords=ds_tile.coords)
    list_of_processed_tiles.append(tile_result_ds)

mh_flux_onset_ds = xr.combine_by_coords(list_of_processed_tiles)

# Save the final dataset
mh_flux_onset_ds.to_netcdf("/g/data/e14/ts7341/Figure_Code_For_Manuscript/Science/melt_during_mhw.nc")

Preparing data...
Getting time coordinates...
Starting tile processing to integrate precursor mh_flux...


Processing Tiles:   0%|          | 0/8 [00:00<?, ?it/s]


--- Reconstructing full dataset from tiles... ---

--- PRECURSOR MELTWATER FLUX ANALYSIS COMPLETE ---
<xarray.Dataset> Size: 10MB
Dimensions:                (lat: 66, lon: 320, event: 57)
Coordinates:
  * lon                    (lon) float64 3kB -59.88 -59.62 ... 19.62 19.88
  * lat                    (lat) float64 528B -69.99 -69.88 ... -63.17 -63.06
  * event                  (event) int64 456B 0 1 2 3 4 5 ... 51 52 53 54 55 56
Data variables:
    precursor_mh_flux_sum  (lat, lon, event) float64 10MB 0.005128 ... nan


### Find the Onset Drivers of the MHWs in the Weddell Sea - this is done for Figs 5 and 6 in the manuscripts

In [39]:
import numpy as np
import xarray as xr
from tqdm.auto import tqdm
from itertools import product

anomaly = anom * 3600 * 24
components = ['Grouped Advection', 'Surface Fluxes Less SW', 'Net Shortwave',
              'Grouped Vertical Mixing', 'entrainment']
surface_flux_components = ["Qprime_1ovrHbar", "Qbar_1ovrHprime", "Qprime_1ovrHprime"]
surface_flux_breakdown = ["lw_heat", "evap_heat", "sens_heat", "mh_flux"]

anom_components_da = anomaly[components + surface_flux_breakdown].to_array(dim='component')
reyn_components_da = ds_reynolds[surface_flux_components].to_array(dim='component')

time_anom_numpy = anom_components_da['time'].values
time_reyn_numpy = reyn_components_da['time'].values

def calculate_onset_drivers_at_point(date_starts, anom_ts_vals, reyn_ts_vals, time_coords_anom, time_coords_reyn):
    nevents = date_starts.shape[0]
    md_out, mv_out = np.full(nevents, np.nan, dtype=object), np.full(nevents, np.nan, dtype=float)
    secd_out, secv_out = np.full(nevents, np.nan, dtype=object), np.full(nevents, np.nan, dtype=float)
    sd_out, sv_out = np.full(nevents, np.nan, dtype=object), np.full(nevents, np.nan, dtype=float)
    bd_out, bv_out = np.full(nevents, np.nan, dtype=object), np.full(nevents, np.nan, dtype=float)
    
    for k in range(nevents):
        event_start = date_starts[k]
        if np.isnat(event_start): continue
        precursor_start = event_start - np.timedelta64(3, 'D')
        
        # Create a mask for the anomaly data
        mask_anom = (time_coords_anom >= precursor_start) & (time_coords_anom <= event_start)
        comp_sums = anom_ts_vals[:len(components), mask_anom].sum(axis=1)
        
        valid_sums = comp_sums[np.isfinite(comp_sums)]
        if valid_sums.size == 0: continue
        original_indices = np.where(np.isfinite(comp_sums))[0]
        sorted_valid_indices = np.argsort(valid_sums)[::-1] # Note: reversed for onset (heating)
        ranked_indices = original_indices[sorted_valid_indices]
        
        md_out[k], mv_out[k] = components[ranked_indices[0]], comp_sums[ranked_indices[0]]
        if len(ranked_indices) > 1:
            secd_out[k], secv_out[k] = components[ranked_indices[1]], comp_sums[ranked_indices[1]]

        ## Figure out what the Reynolds subcomponent is
        if md_out[k] in ['Net Shortwave', 'Surface Fluxes Less SW']:
            # Create a separate, correct mask for the Reynolds data
            mask_reyn = (time_coords_reyn >= precursor_start) & (time_coords_reyn <= event_start)
            sub_sums = reyn_ts_vals[:, mask_reyn].sum(axis=1) # Apply the correct mask
            if np.isfinite(sub_sums).any():
                ranked_sub_indices = np.argsort(sub_sums)[::-1]
                sd_out[k], sv_out[k] = surface_flux_components[ranked_sub_indices[0]], sub_sums[ranked_sub_indices[0]]

        ## Figure out which surface flux
        if md_out[k] == "Surface Fluxes Less SW":
            breakdown_sums = anom_ts_vals[len(components):, mask_anom].sum(axis=1)
            if np.isfinite(breakdown_sums).any():
                b_idx = np.argmax(breakdown_sums)
                bd_out[k], bv_out[k] = surface_flux_breakdown[b_idx], breakdown_sums[b_idx]
                
    return md_out, mv_out, secd_out, secv_out, sd_out, sv_out, bd_out, bv_out

# Tiles again
tile_size_lat = 40 
tile_size_lon = 80 
list_of_processed_tiles = []
lat_starts = range(0, ds.sizes['lat'], tile_size_lat)
lon_starts = range(0, ds.sizes['lon'], tile_size_lon)
tile_combinations = list(product(lat_starts, lon_starts))

for i_start, j_start in tqdm(tile_combinations, desc="Processing Tiles"):
    lat_slice = slice(i_start, i_start + tile_size_lat)
    lon_slice = slice(j_start, j_start + tile_size_lon)
    
    anom_tile_lazy = anom_components_da.isel(lat=lat_slice, lon=lon_slice).transpose('component', 'time', 'lat', 'lon')
    reyn_tile_lazy = reyn_components_da.isel(lat=lat_slice, lon=lon_slice).transpose('component', 'time', 'lat', 'lon')

    ds_tile = ds.isel(lat=lat_slice, lon=lon_slice).load()
    anom_tile = anom_tile_lazy.load()
    reyn_tile = reyn_tile_lazy.load()
    
    tile_nlat, tile_nlon = ds_tile.sizes['lat'], ds_tile.sizes['lon']
    
    tile_md, tile_mv, tile_secd, tile_secv, tile_sd, tile_sv, tile_bd, tile_bv = [
        np.full((tile_nlat, tile_nlon, ds.sizes['event']), np.nan, dtype=dt) 
        for dt in [object, float, object, float, object, float, object, float]
    ]

    for i_tile in range(tile_nlat):
        for j_tile in range(tile_nlon):
            date_starts_point = ds_tile['date_start'].values[i_tile, j_tile, :]
            anom_ts_point = anom_tile.values[:, :, i_tile, j_tile]
            reyn_ts_point = reyn_tile.values[:, :, i_tile, j_tile]
            
            # Pass both time arrays to the worker function
            results = calculate_onset_drivers_at_point(
                date_starts_point, anom_ts_point, reyn_ts_point, 
                time_anom_numpy, time_reyn_numpy
            )
            
            (tile_md[i_tile, j_tile, :], tile_mv[i_tile, j_tile, :], 
             tile_secd[i_tile, j_tile, :], tile_secv[i_tile, j_tile, :],
             tile_sd[i_tile, j_tile, :], tile_sv[i_tile, j_tile, :],
             tile_bd[i_tile, j_tile, :], tile_bv[i_tile, j_tile, :]) = results
    
    tile_result_ds = xr.Dataset(
        data_vars={
            'main_driver': (('lat', 'lon', 'event'), tile_md), 'main_driver_value': (('lat', 'lon', 'event'), tile_mv), 
            'secondary_driver': (('lat', 'lon', 'event'), tile_secd), 'secondary_driver_value': (('lat', 'lon', 'event'), tile_secv),
            'sub_driver': (('lat', 'lon', 'event'), tile_sd), 'sub_driver_value': (('lat', 'lon', 'event'), tile_sv),
            'breakdown_driver': (('lat', 'lon', 'event'), tile_bd), 'breakdown_driver_value': (('lat', 'lon', 'event'), tile_bv),
            'event_start_date': ds_tile['date_start'],
        },
        coords=ds_tile.coords)
    list_of_processed_tiles.append(tile_result_ds)

onset_drivers_ds = xr.combine_by_coords(list_of_processed_tiles)

Preparing data...
Getting time coordinates...
Starting tile processing...


Processing Tiles:   0%|          | 0/8 [00:00<?, ?it/s]

This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the graph without repetition.
See also https://docs.dask.org/en/stable/best-practices.html#load-data-with-dask for more information.
  warnings.warn(

This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the graph without repetition.
See also https://docs.dask.org/en/stable/best-practices.html#load-data-with-dask for more information.
  warnings.warn(

This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the graph without repetition.
See also https://docs.dask.org/en/stable/best-practices.html#load-data-with-dask for more information.
  warnings.warn(

This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the gra


--- Reconstructing full dataset from tiles... ---

--- MHW ONSET DRIVER ANALYSIS COMPLETE ---
<xarray.Dataset> Size: 87MB
Dimensions:                 (lat: 66, lon: 320, event: 57)
Coordinates:
  * lon                     (lon) float64 3kB -59.88 -59.62 ... 19.62 19.88
  * lat                     (lat) float64 528B -69.99 -69.88 ... -63.17 -63.06
  * event                   (event) int64 456B 0 1 2 3 4 5 ... 51 52 53 54 55 56
Data variables:
    main_driver             (lat, lon, event) object 10MB 'Net Shortwave' ......
    main_driver_value       (lat, lon, event) float64 10MB 0.06158 ... nan
    secondary_driver        (lat, lon, event) object 10MB 'entrainment' ... nan
    secondary_driver_value  (lat, lon, event) float64 10MB 0.05685 ... nan
    sub_driver              (lat, lon, event) object 10MB 'Qprime_1ovrHprime'...
    sub_driver_value        (lat, lon, event) float64 10MB 0.01918 1.254 ... nan
    breakdown_driver        (lat, lon, event) object 10MB nan nan ... nan nan
  

In [41]:
## Load and save the data
onset_drivers_ds.load()
onset_drivers_ds.to_netcdf('/scratch/e14/ts7341/mhw_stats_ws.nc')

<xarray.Dataset> Size: 87MB
Dimensions:                 (lat: 66, lon: 320, event: 57)
Coordinates:
  * lon                     (lon) float64 3kB -59.88 -59.62 ... 19.62 19.88
  * lat                     (lat) float64 528B -69.99 -69.88 ... -63.17 -63.06
  * event                   (event) int64 456B 0 1 2 3 4 5 ... 51 52 53 54 55 56
Data variables:
    main_driver             (lat, lon, event) object 10MB 'Net Shortwave' ......
    main_driver_value       (lat, lon, event) float64 10MB 0.06158 ... nan
    secondary_driver        (lat, lon, event) object 10MB 'entrainment' ... nan
    secondary_driver_value  (lat, lon, event) float64 10MB 0.05685 ... nan
    sub_driver              (lat, lon, event) object 10MB 'Qprime_1ovrHprime'...
    sub_driver_value        (lat, lon, event) float64 10MB 0.01918 1.254 ... nan
    breakdown_driver        (lat, lon, event) object 10MB nan nan ... nan nan
    breakdown_driver_value  (lat, lon, event) float64 10MB nan nan ... nan nan
    event_start_date        (lat, lon, event) datetime64[ns] 10MB 1985-10-31T...

In [43]:
onset_drivers = xr.open_dataset('/scratch/e14/ts7341/mhw_stats_ws.nc')
onset_drivers

### Find the Decay Drivers! Not used in manuscript as not unique to sea-ice zones!

In [84]:
import numpy as np
import xarray as xr
from tqdm.auto import tqdm
from itertools import product


anomaly = anom * 3600 * 24
components = ['Grouped Advection', 'Surface Fluxes Less SW', 'Net Shortwave',
              'Grouped Vertical Mixing', 'entrainment']

surface_flux_components = ["Qprime_1ovrHbar", "Qbar_1ovrHprime", "Qprime_1ovrHprime"]
surface_flux_breakdown = ["lw_heat", "evap_heat", "sens_heat", "mh_flux"]

anom_components_da = anomaly[components + surface_flux_breakdown].to_array(dim='component')
reyn_components_da = ds_reynolds[surface_flux_components].to_array(dim='component')

time_anom_numpy = anom_components_da['time'].values
time_reyn_numpy = reyn_components_da['time'].values
-
def calculate_decay_drivers_at_point(date_ends, anom_ts_vals, reyn_ts_vals, time_coords_anom, time_coords_reyn):
    """
    Calculates the primary and secondary COOLING drivers for the decay phase of events.
    """
    nevents = date_ends.shape[0]
    md_out, mv_out = np.full(nevents, np.nan, dtype=object), np.full(nevents, np.nan, dtype=float)
    secd_out, secv_out = np.full(nevents, np.nan, dtype=object), np.full(nevents, np.nan, dtype=float)
    sd_out, sv_out = np.full(nevents, np.nan, dtype=object), np.full(nevents, np.nan, dtype=float)
    bd_out, bv_out = np.full(nevents, np.nan, dtype=object), np.full(nevents, np.nan, dtype=float)
    
    for k in range(nevents):
        event_end = date_ends[k]
        if np.isnat(event_end): continue
        
        # Define the 3-day window LEADING UP to the event's END
        decay_window_start = event_end - np.timedelta64(3, 'D')
        
        mask_anom = (time_coords_anom >= decay_window_start) & (time_coords_anom <= event_end)
        comp_sums = anom_ts_vals[:len(components), mask_anom].sum(axis=1)
        
        valid_sums = comp_sums[np.isfinite(comp_sums)]
        if valid_sums.size == 0: continue
            
        original_indices = np.where(np.isfinite(comp_sums))[0]
    
        sorted_valid_indices = np.argsort(valid_sums)
        ranked_indices = original_indices[sorted_valid_indices]
        
        # The first element [0] is now the index of the most negative value because we want delay
        md_out[k], mv_out[k] = components[ranked_indices[0]], comp_sums[ranked_indices[0]]
        if len(ranked_indices) > 1:
            # The second element [1] is the second most negative value
            secd_out[k], secv_out[k] = components[ranked_indices[1]], comp_sums[ranked_indices[1]]
        
        if md_out[k] in ['Net Shortwave', 'Surface Fluxes Less SW']:
            mask_reyn = (time_coords_reyn >= decay_window_start) & (time_coords_reyn <= event_end)
            sub_sums = reyn_ts_vals[:, mask_reyn].sum(axis=1)
            if np.isfinite(sub_sums).any():
                # Also sort sub-components ascending for cooling
                ranked_sub_indices = np.argsort(sub_sums) 
                sd_out[k], sv_out[k] = surface_flux_components[ranked_sub_indices[0]], sub_sums[ranked_sub_indices[0]]
        
        if md_out[k] == "Surface Fluxes Less SW":
            breakdown_sums = anom_ts_vals[len(components):, mask_anom].sum(axis=1)
            if np.isfinite(breakdown_sums).any():
                b_idx = np.argmin(breakdown_sums)
                bd_out[k], bv_out[k] = surface_flux_breakdown[b_idx], breakdown_sums[b_idx]
                
    return md_out, mv_out, secd_out, secv_out, sd_out, sv_out, bd_out, bv_out

# Tiles...
tile_size_lat = 40 
tile_size_lon = 80 
list_of_processed_tiles = []
lat_starts = range(0, ds.sizes['lat'], tile_size_lat)
lon_starts = range(0, ds.sizes['lon'], tile_size_lon)
tile_combinations = list(product(lat_starts, lon_starts))

for i_start, j_start in tqdm(tile_combinations, desc="Processing Tiles"):
    lat_slice = slice(i_start, i_start + tile_size_lat)
    lon_slice = slice(j_start, j_start + tile_size_lon)
    
    anom_tile_lazy = anom_components_da.isel(lat=lat_slice, lon=lon_slice).transpose('component', 'time', 'lat', 'lon')
    reyn_tile_lazy = reyn_components_da.isel(lat=lat_slice, lon=lon_slice).transpose('component', 'time', 'lat', 'lon')
    
    # Load all data needed for the tile into memory
    ds_tile = ds.isel(lat=lat_slice, lon=lon_slice).load()
    anom_tile = anom_tile_lazy.load()
    reyn_tile = reyn_tile_lazy.load()
    
    tile_nlat, tile_nlon = ds_tile.sizes['lat'], ds_tile.sizes['lon']
    
    # Output
    tile_md_decay, tile_mv_decay, tile_secd_decay, tile_secv_decay, tile_sd_decay, tile_sv_decay, tile_bd_decay, tile_bv_decay = [
        np.full((tile_nlat, tile_nlon, ds.sizes['event']), np.nan, dtype=dt) 
        for dt in [object, float, object, float, object, float, object, float]
    ]

    for i_tile in range(tile_nlat):
        for j_tile in range(tile_nlon):
            # Date end is important to use, we are looking at decay
            date_ends_point = ds_tile['date_end'].values[i_tile, j_tile, :]
            anom_ts_point = anom_tile.values[:, :, i_tile, j_tile]
            reyn_ts_point = reyn_tile.values[:, :, i_tile, j_tile]
            
            results = calculate_decay_drivers_at_point(
                date_ends_point, anom_ts_point, reyn_ts_point, 
                time_anom_numpy, time_reyn_numpy
            )
            
            (tile_md_decay[i_tile, j_tile, :], tile_mv_decay[i_tile, j_tile, :], 
             tile_secd_decay[i_tile, j_tile, :], tile_secv_decay[i_tile, j_tile, :],
             tile_sd_decay[i_tile, j_tile, :], tile_sv_decay[i_tile, j_tile, :],
             tile_bd_decay[i_tile, j_tile, :], tile_bv_decay[i_tile, j_tile, :]) = results
    
    tile_result_ds = xr.Dataset(
        data_vars={
            'main_decay_driver': (('lat', 'lon', 'event'), tile_md_decay),
            'main_decay_value': (('lat', 'lon', 'event'), tile_mv_decay), 
            'secondary_decay_driver': (('lat', 'lon', 'event'), tile_secd_decay),
            'secondary_decay_value': (('lat', 'lon', 'event'), tile_secv_decay),
            'sub_decay_driver': (('lat', 'lon', 'event'), tile_sd_decay),
            'sub_decay_value': (('lat', 'lon', 'event'), tile_sv_decay),
            'breakdown_decay_driver': (('lat', 'lon', 'event'), tile_bd_decay),
            'breakdown_decay_value': (('lat', 'lon', 'event'), tile_bv_decay),
            'event_end_date': ds_tile['date_end'], # Save the end date
        },
        coords=ds_tile.coords)
    list_of_processed_tiles.append(tile_result_ds)

decay_drivers_ds = xr.combine_by_coords(list_of_processed_tiles)

Preparing data for DECAY analysis...
Getting time coordinates...
Starting tile processing for DECAY drivers...


Processing Tiles:   0%|          | 0/8 [00:00<?, ?it/s]

This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the graph without repetition.
See also https://docs.dask.org/en/stable/best-practices.html#load-data-with-dask for more information.
  warnings.warn(

This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the graph without repetition.
See also https://docs.dask.org/en/stable/best-practices.html#load-data-with-dask for more information.
  warnings.warn(

This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the graph without repetition.
See also https://docs.dask.org/en/stable/best-practices.html#load-data-with-dask for more information.
  warnings.warn(

This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the gra


--- Reconstructing full dataset from tiles... ---

--- MHW DECAY DRIVER ANALYSIS COMPLETE ---
<xarray.Dataset> Size: 81MB
Dimensions:                 (lat: 66, lon: 320, event: 53)
Coordinates:
  * lon                     (lon) float64 3kB -59.88 -59.62 ... 19.62 19.88
  * lat                     (lat) float64 528B -69.99 -69.88 ... -63.17 -63.06
  * event                   (event) int64 424B 4 5 6 7 8 9 ... 51 52 53 54 55 56
Data variables:
    main_decay_driver       (lat, lon, event) object 9MB nan nan nan ... nan nan
    main_decay_value        (lat, lon, event) float64 9MB nan nan ... nan nan
    secondary_decay_driver  (lat, lon, event) object 9MB nan nan nan ... nan nan
    secondary_decay_value   (lat, lon, event) float64 9MB nan nan ... nan nan
    sub_decay_driver        (lat, lon, event) object 9MB nan nan nan ... nan nan
    sub_decay_value         (lat, lon, event) float64 9MB nan nan ... nan nan
    breakdown_decay_driver  (lat, lon, event) object 9MB nan nan nan ... nan

In [85]:
decay_drivers_ds.load()
decay_drivers_ds.to_netcdf('/scratch/e14/ts7341/mhw_decay_ws.nc')

<xarray.Dataset> Size: 81MB
Dimensions:                 (lat: 66, lon: 320, event: 53)
Coordinates:
  * lon                     (lon) float64 3kB -59.88 -59.62 ... 19.62 19.88
  * lat                     (lat) float64 528B -69.99 -69.88 ... -63.17 -63.06
  * event                   (event) int64 424B 4 5 6 7 8 9 ... 51 52 53 54 55 56
Data variables:
    main_decay_driver       (lat, lon, event) object 9MB nan nan nan ... nan nan
    main_decay_value        (lat, lon, event) float64 9MB nan nan ... nan nan
    secondary_decay_driver  (lat, lon, event) object 9MB nan nan nan ... nan nan
    secondary_decay_value   (lat, lon, event) float64 9MB nan nan ... nan nan
    sub_decay_driver        (lat, lon, event) object 9MB nan nan nan ... nan nan
    sub_decay_value         (lat, lon, event) float64 9MB nan nan ... nan nan
    breakdown_decay_driver  (lat, lon, event) object 9MB nan nan nan ... nan nan
    breakdown_decay_value   (lat, lon, event) float64 9MB nan nan ... nan nan
    event_end_date          (lat, lon, event) datetime64[ns] 9MB NaT NaT ... NaT

## Case Study 2: Ross Sea

In [33]:
ds = xr.open_dataset('/g/data/e14/ts7341/Figure_Code_For_Manuscript/Science/full_MHW_data_1982_2024_RS.nc')

In [34]:
## Unused – was not filtered for 2014 onward, that was an old analyses

# # Filter for 2014 – this is what we did for the figure in the appendix that was examining the generalised onset drivers, Ross Sea lows persist since then (high transport – Krauzig et al. (2014))
# condition = ds['date_start'].dt.year >= 2014
# ds_filtered = ds.where(condition, drop=True)

# # Count the number of non-NaN events along the 'event' dimension
# new_event_count = ds_filtered['index_start'].count(dim='event')

# # Assign the updated count back into the dataset
# ds_filtered['n_event'] = new_event_count

# print(ds_filtered)

Filtering dataset for year >= 2014...
Recalculating n_event...

Final Filtered Dataset:
<xarray.Dataset> Size: 69MB
Dimensions:          (lat: 66, lon: 278, event: 47)
Coordinates:
  * lon              (lon) float64 2kB -199.4 -199.1 -198.9 ... -130.4 -130.1
  * lat              (lat) float64 528B -76.96 -76.85 -76.75 ... -70.2 -70.09
  * event            (event) int64 376B 3 4 5 6 7 8 9 ... 43 44 45 46 47 48 49
Data variables:
    index_start      (lat, lon, event) float64 7MB nan nan nan ... nan nan nan
    index_end        (lat, lon, event) float64 7MB nan nan nan ... nan nan nan
    date_start       (lat, lon, event) datetime64[ns] 7MB NaT NaT ... NaT NaT
    date_end         (lat, lon, event) datetime64[ns] 7MB NaT NaT ... NaT NaT
    duration         (lat, lon, event) float64 7MB nan nan nan ... nan nan nan
    n_event          (lat, lon) int64 147kB 0 0 0 0 0 0 0 0 ... 2 2 1 1 1 2 2 2
    index_peak       (lat, lon, event) float64 7MB nan nan nan ... nan nan nan
    date_peak   

In [35]:
ds = ds_filtered.transpose('lat', 'lon', 'event')
ds = ds.sel(lat = yt_coord, lon = xt_coord)
ds

<xarray.Dataset> Size: 69MB
Dimensions:          (lat: 66, lon: 278, event: 47)
Coordinates:
  * lon              (lon) float64 2kB -199.4 -199.1 -198.9 ... -130.4 -130.1
  * lat              (lat) float64 528B -76.96 -76.85 -76.75 ... -70.2 -70.09
  * event            (event) int64 376B 3 4 5 6 7 8 9 ... 43 44 45 46 47 48 49
Data variables:
    index_start      (lat, lon, event) float64 7MB nan nan nan ... nan nan nan
    index_end        (lat, lon, event) float64 7MB nan nan nan ... nan nan nan
    date_start       (lat, lon, event) datetime64[ns] 7MB NaT NaT ... NaT NaT
    date_end         (lat, lon, event) datetime64[ns] 7MB NaT NaT ... NaT NaT
    duration         (lat, lon, event) float64 7MB nan nan nan ... nan nan nan
    n_event          (lat, lon) int64 147kB 0 0 0 0 0 0 0 0 ... 2 2 1 1 1 2 2 2
    index_peak       (lat, lon, event) float64 7MB nan nan nan ... nan nan nan
    date_peak        (lat, lon, event) datetime64[ns] 7MB NaT NaT ... NaT NaT
    intensity_max    (lat, lon, event) float64 7MB nan nan nan ... nan nan nan
    intensity_mean   (lat, lon, event) float64 7MB nan nan nan ... nan nan nan
    intensity_cumul  (lat, lon, event) float64 7MB nan nan nan ... nan nan nan

In [36]:
ds = ds.load()

In [37]:
anom = anom.rename({'xt_ocean':'lon','yt_ocean':'lat'})

In [38]:
anom = anom.chunk({'time':-1})

In [39]:
anom

<xarray.Dataset> Size: 15GB
Dimensions:                  (lon: 280, lat: 66, time: 3642)
Coordinates:
  * lon                      (lon) float64 2kB -199.9 -199.6 ... -130.4 -130.1
  * lat                      (lat) float64 528B -76.96 -76.85 ... -70.2 -70.09
  * time                     (time) datetime64[ns] 29kB 2013-12-31T12:00:00 ....
    dayofyear                (time) int64 29kB dask.array<chunksize=(3642,), meta=np.ndarray>
Data variables: (12/28)
    fixedh_tendency          (time, lat, lon) float64 538MB dask.array<chunksize=(3642, 66, 280), meta=np.ndarray>
    temp_adv                 (time, lat, lon) float64 538MB dask.array<chunksize=(3642, 66, 280), meta=np.ndarray>
    submeso                  (time, lat, lon) float64 538MB dask.array<chunksize=(3642, 66, 280), meta=np.ndarray>
    neut_diffusion           (time, lat, lon) float64 538MB dask.array<chunksize=(3642, 66, 280), meta=np.ndarray>
    neutral_gm               (time, lat, lon) float64 538MB dask.array<chunksize=(3642, 66, 280), meta=np.ndarray>
    temp_vdiffuse_k33        (time, lat, lon) float64 538MB dask.array<chunksize=(3642, 66, 280), meta=np.ndarray>
    ...                       ...
    Grouped Vertical Mixing  (time, lat, lon) float64 538MB dask.array<chunksize=(3642, 66, 280), meta=np.ndarray>
    Surface Fluxes Grouped   (time, lat, lon) float64 538MB dask.array<chunksize=(3642, 66, 280), meta=np.ndarray>
    Surface Fluxes Less SW   (time, lat, lon) float64 538MB dask.array<chunksize=(3642, 66, 280), meta=np.ndarray>
    mh_flux                  (time, lat, lon) float64 538MB dask.array<chunksize=(3642, 66, 280), meta=np.ndarray>
    Shortwave Penetration    (time, lat, lon) float64 538MB dask.array<chunksize=(3642, 66, 280), meta=np.ndarray>
    Net Shortwave            (time, lat, lon) float64 538MB dask.array<chunksize=(3642, 66, 280), meta=np.ndarray>

In [40]:
ds_reynolds = ds_reynolds.sel(time = slice('2013-12-31','2023-12-31'))

In [41]:
ds_reynolds

<xarray.Dataset> Size: 2GB
Dimensions:             (xt_ocean: 280, yt_ocean: 66, time: 3650)
Coordinates:
  * xt_ocean            (xt_ocean) float64 2kB -199.9 -199.6 ... -130.4 -130.1
  * yt_ocean            (yt_ocean) float64 528B -76.96 -76.85 ... -70.2 -70.09
  * time                (time) datetime64[ns] 29kB 2014-01-01T12:00:00 ... 20...
    dayofyear           (time) int64 29kB 1 2 3 4 5 6 ... 359 360 361 362 363
Data variables:
    QoverH              (time, yt_ocean, xt_ocean) float32 270MB dask.array<chunksize=(363, 66, 160), meta=np.ndarray>
    Qbar_1ovrHbar       (yt_ocean, xt_ocean, time) float32 270MB dask.array<chunksize=(3, 7, 183), meta=np.ndarray>
    Qprime_1ovrHbar     (time, yt_ocean, xt_ocean) float32 270MB dask.array<chunksize=(183, 3, 7), meta=np.ndarray>
    Qbar_1ovrHprime     (yt_ocean, xt_ocean, time) float32 270MB dask.array<chunksize=(3, 7, 183), meta=np.ndarray>
    Qprime_1ovrHprime   (time, yt_ocean, xt_ocean) float32 270MB dask.array<chunksize=(183, 3, 7), meta=np.ndarray>
    Residual            (time, yt_ocean, xt_ocean) float32 270MB dask.array<chunksize=(183, 3, 7), meta=np.ndarray>
    Residual_Corrected  (time, yt_ocean, xt_ocean) float32 270MB dask.array<chunksize=(183, 3, 7), meta=np.ndarray>

### Ross Sea Decay Drivers! Important for making Fig 9

In [42]:
import numpy as np
import xarray as xr
from tqdm.auto import tqdm
from itertools import product

anomaly = anom * 3600 * 24
components = ['Grouped Advection', 'Surface Fluxes Less SW', 'Net Shortwave',
              'Grouped Vertical Mixing', 'entrainment']
surface_flux_breakdown = ["lw_heat", "evap_heat", "sens_heat"]

anom_components_da = anomaly[components + surface_flux_breakdown].to_array(dim='component').transpose('component', 'time', 'lat', 'lon')
time_anom_numpy = anom_components_da['time'].values

def calculate_decay_drivers_at_point(date_ends, anom_ts_vals, time_coords):
    nevents = date_ends.shape[0]
    md_out, mv_out = np.full(nevents, np.nan, dtype=object), np.full(nevents, np.nan, dtype=float)
    secd_out, secv_out = np.full(nevents, np.nan, dtype=object), np.full(nevents, np.nan, dtype=float)
    bd_out, bv_out = np.full(nevents, np.nan, dtype=object), np.full(nevents, np.nan, dtype=float)
    
    for k in range(nevents):
        event_end = date_ends[k]  # Use the end date
        if np.isnat(event_end): continue
        
        # This defines the 3-day window LEADING UP TO the end of the event
        decay_period_start = event_end - np.timedelta64(3, 'D')
        mask = (time_coords >= decay_period_start) & (time_coords <= event_end)
        comp_sums = anom_ts_vals[:len(components), mask].sum(axis=1)
        
        valid_sums = comp_sums[np.isfinite(comp_sums)]
        if valid_sums.size == 0:
            continue

        original_indices = np.where(np.isfinite(comp_sums))[0]
        sorted_valid_indices = np.argsort(valid_sums) # Finds the most negative (cooling) terms
        ranked_indices = original_indices[sorted_valid_indices]
        
        md_out[k] = components[ranked_indices[0]]
        mv_out[k] = comp_sums[ranked_indices[0]]
        
        if len(ranked_indices) > 1:
            secd_out[k] = components[ranked_indices[1]]
            secv_out[k] = comp_sums[ranked_indices[1]]
        
        if md_out[k] == "Surface Fluxes Less SW":
            breakdown_sums = anom_ts_vals[len(components):, mask].sum(axis=1)
            if np.isfinite(breakdown_sums).any():
                b_idx = np.argmin(breakdown_sums)
                bd_out[k], bv_out[k] = surface_flux_breakdown[b_idx], breakdown_sums[b_idx]
                
    return md_out, mv_out, secd_out, secv_out, bd_out, bv_out

# New tiles
tile_size_lat = 36
tile_size_lon = 70
list_of_processed_tiles = []
lat_starts = range(0, ds.sizes['lat'], tile_size_lat)
lon_starts = range(0, ds.sizes['lon'], tile_size_lon)
tile_combinations = list(product(lat_starts, lon_starts))

for i_start, j_start in tqdm(tile_combinations, desc="Processing Tiles"):
    lat_slice = slice(i_start, i_start + tile_size_lat)
    lon_slice = slice(j_start, j_start + tile_size_lon)
    
    # Load data for the current tile
    ds_tile = ds.isel(lat=lat_slice, lon=lon_slice).load()
    anom_tile = anom_components_da.isel(lat=lat_slice, lon=lon_slice).load()
    
    tile_nlat, tile_nlon = ds_tile.sizes['lat'], ds_tile.sizes['lon']
    
    tile_md, tile_mv = np.full((tile_nlat, tile_nlon, ds.sizes['event']), np.nan, dtype=object), np.full((tile_nlat, tile_nlon, ds.sizes['event']), np.nan, dtype=float)
    tile_secd, tile_secv = np.full((tile_nlat, tile_nlon, ds.sizes['event']), np.nan, dtype=object), np.full((tile_nlat, tile_nlon, ds.sizes['event']), np.nan, dtype=float)
    tile_bd, tile_bv = np.full((tile_nlat, tile_nlon, ds.sizes['event']), np.nan, dtype=object), np.full((tile_nlat, tile_nlon, ds.sizes['event']), np.nan, dtype=float)

    for i_tile in range(tile_nlat):
        for j_tile in range(tile_nlon):
            date_ends_point = ds_tile['date_end'].values[i_tile, j_tile, :]
            anom_ts_point = anom_tile.values[:, :, i_tile, j_tile]
            
            md, mv, secd, secv, bd, bv = calculate_decay_drivers_at_point(
                date_ends_point, anom_ts_point, time_anom_numpy
            )
            
            tile_md[i_tile, j_tile, :] = md; tile_mv[i_tile, j_tile, :] = mv
            tile_secd[i_tile, j_tile, :] = secd; tile_secv[i_tile, j_tile, :] = secv
            tile_bd[i_tile, j_tile, :] = bd; tile_bv[i_tile, j_tile, :] = bv
    
    tile_result_ds = xr.Dataset(
        data_vars={
            'main_decay_driver': (('lat', 'lon', 'event'), tile_md),
            'main_decay_value': (('lat', 'lon', 'event'), tile_mv),
            'secondary_decay_driver': (('lat', 'lon', 'event'), tile_secd),
            'secondary_decay_value': (('lat', 'lon', 'event'), tile_secv),
            'breakdown_driver': (('lat', 'lon', 'event'), tile_bd),
            'breakdown_driver_value': (('lat', 'lon', 'event'), tile_bv),
            'event_end_date': ds_tile['date_end'], # Save the end date for filtering
        },
        coords=ds_tile.coords
    )
    list_of_processed_tiles.append(tile_result_ds)
    
per_event_drivers_ds = xr.combine_by_coords(list_of_processed_tiles)

Starting tile processing...


Processing Tiles:   0%|          | 0/8 [00:00<?, ?it/s]

This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the graph without repetition.
See also https://docs.dask.org/en/stable/best-practices.html#load-data-with-dask for more information.
  warnings.warn(

2025-10-29 12:39:40,776 - distributed.worker - ERROR - Failed to communicate with scheduler during heartbeat.
Traceback (most recent call last):
  File "/g/data/xp65/public/apps/med_conda/envs/analysis3-24.12/lib/python3.11/site-packages/distributed/comm/tcp.py", line 225, in read
    frames_nosplit_nbytes_bin = await stream.read_bytes(fmt_size)
                                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
tornado.iostream.StreamClosedError: Stream is closed

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/g/data/xp65/public/apps/med_conda/envs/analysis3-24.12/lib/python3.11/site-packages/distributed/worker.py", line 1269, in heartbeat


In [ ]:
per_event_drivers_ds.load()

In [23]:
per_event_drivers_ds.to_netcdf('/scratch/e14/ts7341/mhw_decay_Ross_Sea.nc')
print("Save complete.")

Save complete.


In [23]:
ds = xr.open_dataset('/scratch/e14/ts7341/mhw_decay_Ross_Sea.nc')
ds

<xarray.Dataset> Size: 203MB
Dimensions:                 (lat: 66, lon: 278, event: 44)
Coordinates:
  * lon                     (lon) float64 2kB -199.4 -199.1 ... -130.4 -130.1
  * lat                     (lat) float64 528B -76.96 -76.85 ... -70.2 -70.09
  * event                   (event) int64 352B 0 1 2 3 4 5 ... 38 39 40 41 42 43
Data variables:
    main_decay_driver       (lat, lon, event) <U23 74MB ...
    main_decay_value        (lat, lon, event) float64 6MB ...
    secondary_decay_driver  (lat, lon, event) <U23 74MB ...
    secondary_decay_value   (lat, lon, event) float64 6MB ...
    breakdown_driver        (lat, lon, event) <U9 29MB ...
    breakdown_driver_value  (lat, lon, event) float64 6MB ...
    event_start_date        (lat, lon, event) datetime64[ns] 6MB ...

### Onset Drivers – We did this and then only selected two years per Fig A10

In [47]:
import numpy as np
import xarray as xr
from tqdm.auto import tqdm
from itertools import product

anomaly = anom * 3600 * 24
components = ['Grouped Advection', 'Surface Fluxes Less SW', 'Net Shortwave',
              'Grouped Vertical Mixing', 'entrainment']
surface_flux_breakdown = ["lw_heat", "evap_heat", "sens_heat", "mh_flux"]

anom_components_da = anomaly[components + surface_flux_breakdown].to_array(dim='component').transpose('component', 'time', 'lat', 'lon')
time_anom_numpy = anom_components_da['time'].values

def calculate_onset_drivers_at_point(date_starts, anom_ts_vals, time_coords):
    """
    Calculates the primary and secondary ONSET drivers for the start of events.
    """
    nevents = date_starts.shape[0]
    mo_out, mv_out = np.full(nevents, np.nan, dtype=object), np.full(nevents, np.nan, dtype=float)
    seco_out, secov_out = np.full(nevents, np.nan, dtype=object), np.full(nevents, np.nan, dtype=float)
    bd_out, bv_out = np.full(nevents, np.nan, dtype=object), np.full(nevents, np.nan, dtype=float)
    
    for k in range(nevents):
        event_start_date = date_starts[k]
        if np.isnat(event_start_date): 
            continue
        
        event_end_period = event_start_date - np.timedelta64(3, 'D')
        mask = (time_coords >= event_start_date) & (time_coords <= event_end_period)
        
        if not np.any(mask):
            continue

        comp_sums = anom_ts_vals[:len(components), mask].sum(axis=1)
        
        valid_sums = comp_sums[np.isfinite(comp_sums)]
        if valid_sums.size == 0:
            continue

        original_indices = np.where(np.isfinite(comp_sums))[0]
        # Sort ascending, then take the last indices to get the most POSITIVE terms
        sorted_valid_indices = np.argsort(valid_sums) 
        ranked_indices = original_indices[sorted_valid_indices]
        
        # Find the largest positive value 
        mo_out[k] = components[ranked_indices[-1]]
        mv_out[k] = comp_sums[ranked_indices[-1]]
        
        if len(ranked_indices) > 1:
            # Find the second largest positive value
            seco_out[k] = components[ranked_indices[-2]]
            secov_out[k] = comp_sums[ranked_indices[-2]]
        
        # Check if the main driver was surface fluxes
        if mo_out[k] == "Surface Fluxes Less SW":
            breakdown_sums = anom_ts_vals[len(components):, mask].sum(axis=1)
            if np.isfinite(breakdown_sums).any():
                # Find the index of the maximum (most positive) flux
                b_idx = np.nanargmax(breakdown_sums) # Use nanargmax for safety
                bd_out[k], bv_out[k] = surface_flux_breakdown[b_idx], breakdown_sums[b_idx]
                
    return mo_out, mv_out, seco_out, secov_out, bd_out, bv_out

# Tiles
tile_size_lat = 36
tile_size_lon = 70
list_of_processed_tiles = []
lat_starts = range(0, ds.sizes['lat'], tile_size_lat)
lon_starts = range(0, ds.sizes['lon'], tile_size_lon)
tile_combinations = list(product(lat_starts, lon_starts))

print("Starting tile processing for ONSET drivers...")
for i_start, j_start in tqdm(tile_combinations, desc="Processing Tiles"):
    lat_slice = slice(i_start, i_start + tile_size_lat)
    lon_slice = slice(j_start, j_start + tile_size_lon)
    
    # Load data for the current tile
    ds_tile = ds.isel(lat=lat_slice, lon=lon_slice).load()
    anom_tile = anom_components_da.isel(lat=lat_slice, lon=lon_slice).load()
    
    tile_nlat, tile_nlon = ds_tile.sizes['lat'], ds_tile.sizes['lon']
    

    tile_mo, tile_mv = np.full((tile_nlat, tile_nlon, ds.sizes['event']), np.nan, dtype=object), np.full((tile_nlat, tile_nlon, ds.sizes['event']), np.nan, dtype=float)
    tile_seco, tile_secov = np.full((tile_nlat, tile_nlon, ds.sizes['event']), np.nan, dtype=object), np.full((tile_nlat, tile_nlon, ds.sizes['event']), np.nan, dtype=float)
    tile_bd, tile_bv = np.full((tile_nlat, tile_nlon, ds.sizes['event']), np.nan, dtype=object), np.full((tile_nlat, tile_nlon, ds.sizes['event']), np.nan, dtype=float)

    for i_tile in range(tile_nlat):
        for j_tile in range(tile_nlon):
            
            date_starts_point = ds_tile['date_start'].values[i_tile, j_tile, :]
            anom_ts_point = anom_tile.values[:, :, i_tile, j_tile]
            
            # Call the modified function
            mo, mv, seco, secov, bd, bv = calculate_onset_drivers_at_point(
                date_starts_point, anom_ts_point, time_anom_numpy
            )
            
            tile_mo[i_tile, j_tile, :] = mo; tile_mv[i_tile, j_tile, :] = mv
            tile_seco[i_tile, j_tile, :] = seco; tile_secov[i_tile, j_tile, :] = secov
            tile_bd[i_tile, j_tile, :] = bd; tile_bv[i_tile, j_tile, :] = bv
    
    tile_result_ds = xr.Dataset(
        data_vars={
            'main_onset_driver': (('lat', 'lon', 'event'), tile_mo),
            'main_onset_value': (('lat', 'lon', 'event'), tile_mv),
            'secondary_onset_driver': (('lat', 'lon', 'event'), tile_seco),
            'secondary_onset_value': (('lat', 'lon', 'event'), tile_secov),
            'breakdown_driver': (('lat', 'lon', 'event'), tile_bd),
            'breakdown_driver_value': (('lat', 'lon', 'event'), tile_bv),
            'event_start_date': ds_tile['date_start'], # Save the start date for filtering
        },
        coords=ds_tile.coords
    )
    list_of_processed_tiles.append(tile_result_ds)

per_event_drivers_ds = xr.combine_by_coords(list_of_processed_tiles)

Starting tile processing for ONSET drivers...


Processing Tiles:   0%|          | 0/8 [00:00<?, ?it/s]

This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the graph without repetition.
See also https://docs.dask.org/en/stable/best-practices.html#load-data-with-dask for more information.
  warnings.warn(




--- Reconstructing full dataset from tiles... ---

--- MASTER ONSET DRIVER DATASET COMPLETE ---
<xarray.Dataset> Size: 48MB
Dimensions:                 (lat: 66, lon: 278, event: 47)
Coordinates:
  * lon                     (lon) float64 2kB -199.4 -199.1 ... -130.4 -130.1
  * lat                     (lat) float64 528B -76.96 -76.85 ... -70.2 -70.09
  * event                   (event) int64 376B 3 4 5 6 7 8 ... 44 45 46 47 48 49
Data variables:
    main_onset_driver       (lat, lon, event) object 7MB nan nan nan ... nan nan
    main_onset_value        (lat, lon, event) float64 7MB nan nan ... nan nan
    secondary_onset_driver  (lat, lon, event) object 7MB nan nan nan ... nan nan
    secondary_onset_value   (lat, lon, event) float64 7MB nan nan ... nan nan
    breakdown_driver        (lat, lon, event) object 7MB nan nan nan ... nan nan
    breakdown_driver_value  (lat, lon, event) float64 7MB nan nan ... nan nan
    event_start_date        (lat, lon, event) datetime64[ns] 7MB NaT NaT 

In [48]:
per_event_drivers_ds.load()

<xarray.Dataset> Size: 48MB
Dimensions:                 (lat: 66, lon: 278, event: 47)
Coordinates:
  * lon                     (lon) float64 2kB -199.4 -199.1 ... -130.4 -130.1
  * lat                     (lat) float64 528B -76.96 -76.85 ... -70.2 -70.09
  * event                   (event) int64 376B 3 4 5 6 7 8 ... 44 45 46 47 48 49
Data variables:
    main_onset_driver       (lat, lon, event) object 7MB nan nan nan ... nan nan
    main_onset_value        (lat, lon, event) float64 7MB nan nan ... nan nan
    secondary_onset_driver  (lat, lon, event) object 7MB nan nan nan ... nan nan
    secondary_onset_value   (lat, lon, event) float64 7MB nan nan ... nan nan
    breakdown_driver        (lat, lon, event) object 7MB nan nan nan ... nan nan
    breakdown_driver_value  (lat, lon, event) float64 7MB nan nan ... nan nan
    event_start_date        (lat, lon, event) datetime64[ns] 7MB NaT NaT ... NaT

In [49]:
per_event_drivers_ds.to_netcdf('/scratch/e14/ts7341/mhw_onset_Ross_Sea.nc')
print("Save complete.")

Save complete.
